In [17]:
import os
import cv2
import xml.etree.ElementTree as ET

def load_dataset(dataset_dir):
    images = []
    annotations = []
    for filename in os.listdir(dataset_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(dataset_dir, filename)
            annotation_path = os.path.join(dataset_dir, filename[:-4] + '.xml')
            images.append(image_path)
            annotations.append(annotation_path)
    return images, annotations

In [18]:
def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image.astype('float32')
    image /= 255.0  # Normalisasi nilai piksel menjadi [0, 1]
    return image

In [19]:
def process_annotation(annotation_path, target_size):
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    boxes = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        box = obj.find('bndbox')
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)

        # Ubah koordinat kotak pembatas sesuai dengan ukuran gambar yang diubah
        xmin = xmin * target_size[1] / original_width
        ymin = ymin * target_size[0] / original_height
        xmax = xmax * target_size[1] / original_width
        ymax = ymax * target_size[0] / original_height

        boxes.append([xmin, ymin, xmax, ymax, label])

    return boxes

In [20]:
import tensorflow as tf

def create_tf_example(image_path, annotation_path, target_size):
    image = preprocess_image(image_path, target_size)
    boxes = process_annotation(annotation_path, target_size)

    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tobytes()])),
        'xmin': tf.train.Feature(float_list=tf.train.FloatList(value=[box[0] for box in boxes])),
        'ymin': tf.train.Feature(float_list=tf.train.FloatList(value=[box[1] for box in boxes])),
        'xmax': tf.train.Feature(float_list=tf.train.FloatList(value=[box[2] for box in boxes])),
        'ymax': tf.train.Feature(float_list=tf.train.FloatList(value=[box[3] for box in boxes])),
        'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[box[4].encode() for box in boxes]))
    }

    example = tf.train.Example(features=tf.train.Features(feature=feature))
    return example


In [21]:
# Definisikan konfigurasi model seperti jumlah kelas dan ukuran gambar input
num_classes = 1
input_size = (300, 300)

In [22]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load dataset
train_images, train_annotations = load_dataset('train/')
test_images, test_annotations = load_dataset('test/')

# Create and compile the model
num_classes = 2
input_size = (300, 300)
base_model = MobileNetV2(input_shape=input_size + (3,), include_top=False, weights='imagenet')
x = Flatten()(base_model.output)
output = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Preprocess data and create TFRecords
# ...

# Load TFRecords and train the model
batch_size = 32
num_epochs = 10

# Convert training annotations to class indices
train_labels = []
for annotation_path in train_annotations:
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    for obj in root.findall('object'):
        label = obj.find('name').text
        label_idx = 0 if label == 'melon' else 1  # Assuming 'melon' is the first class and 'background' is the second class
        train_labels.append(label_idx)

# Convert test annotations to class indices
test_labels = []
for annotation_path in test_annotations:
    tree = ET.parse(annotation_path)
    root = tree.getroot()
    for obj in root.findall('object'):
        label = obj.find('name').text
        label_idx = 0 if label == 'melon' else 1  # Assuming 'melon' is the first class and 'background' is the second class
        test_labels.append(label_idx)

# Create input pipeline for training data
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(parse_function).shuffle(buffer_size=1000).batch(batch_size).repeat()

# Create input pipeline for test data
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(parse_function).batch(batch_size)

# Define the parse_function to load and preprocess images and labels
def parse_function(image_path, label):
    image = preprocess_image(image_path, input_size)
    return image, label

# Train the model
steps_per_epoch = len(train_images) // batch_size
validation_steps = len(test_images) // batch_size

model.fit(
    train_dataset,
    epochs=num_epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps
)

# Save the trained model
model.save('model.h5')


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'train/'